In [1]:
import os, sys
import numpy as np
sys.path.append('../')

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
from onsstove.onsstove import OnSSTOVE
from onsstove.layer import RasterLayer, VectorLayer
from onsstove.raster import interpolate

In [4]:
nepal = OnSSTOVE(project_crs=3857, cell_size=(1000, 1000))
output_directory = 'results'
nepal.output_directory = output_directory

In [5]:
path = r"C:\Users\camilorg\Box Sync\Clean cooking Africa paper\05. OnSSTOVE test case\GQ_test_file_social_specs.csv"
nepal.read_scenario_data(path)

In [6]:
path = r"C:\Users\camilorg\Box Sync\Clean cooking Africa paper\05. OnSSTOVE test case\GQ_test_file_tech_specs.csv"
nepal.read_tech_data(path)

In [7]:
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Administrative\npl_admbnda_nd_20201117_shp\npl_admbnda_adm0_nd_20201117.shp"
mask_layer = VectorLayer('vector', 'adm_0', layer_path=path)
os.makedirs(f'{output_directory}/vector/adm_0', exist_ok=True)
mask_layer.reproject(3857, f'{output_directory}/vector/adm_0')
nepal.mask_layer = mask_layer
# add admin_1 or 2 or 3 layer to gdf here

In [8]:
path = r"data\population_npl_2018-10-01.tif"
os.makedirs(f'{output_directory}/rasters/population', exist_ok=True)
population = RasterLayer('rasters', 'population', layer_path=path, resample='sum')
population.reproject(3857, f'{output_directory}/rasters/population', 1000, 1000)
population.mask(nepal.mask_layer.layer, f'{output_directory}/rasters/population')
nepal.base_layer = population
nepal.population_to_dataframe(population)

In [9]:
np.nansum(population.layer)

28452301.739626173

In [10]:
nepal.calibrate_pop()

In [11]:
nepal.calibrate_urban()

In [12]:
rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc = nepal.base_fuel.relative_risk()

In [13]:
print(rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc)

3.379491698111922 2.6016762384532757 2.9461880136695875 3.807237414338239


In [14]:
paf_0_alri = nepal.base_fuel.paf(rr_0_alri, 1 - nepal.specs['clean_cooking_access'])
paf_0_copd = nepal.base_fuel.paf(rr_0_copd, 1 - nepal.specs['clean_cooking_access'])
paf_0_lc  = nepal.base_fuel.paf(rr_0_ihd, 1 - nepal.specs['clean_cooking_access'])
paf_0_ihd = nepal.base_fuel.paf(rr_0_lc, 1 - nepal.specs['clean_cooking_access'])

In [15]:
nepal.base_fuel.carb(nepal.gdf)

In [16]:
friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_walking_only_friction_surface\2020_walking_only_friction_surface.geotiff"
forest_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\Forest cover\Global Forest Cover Change (GFCC).tif"

nepal.base_fuel.total_time(nepal.specs, friction_path, forest_path, population.path, output_directory)

In [17]:
nepal.raster_to_dataframe(nepal.base_fuel.total_time_yr, name='base_fuel_time', method='read')

In [18]:
nepal.raster_to_dataframe(nepal.base_fuel.travel_time, name='base_fuel_travel_time', method='read')

In [19]:
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\Nepal_DL0\Nepal_DL0.shp"
mv_lines = VectorLayer('vector', 'MV_line', layer_path=path)
os.makedirs(f'{output_directory}/vector/MV_line', exist_ok=True)
mv_lines.reproject(3857, f'{output_directory}/vector/MV_line')

path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\HV-network\HV_lines_georeferenced.shp"
hv_lines = VectorLayer('vector', 'HV_line', layer_path=path)
os.makedirs(f'{output_directory}/vector/HV_line', exist_ok=True)
hv_lines.reproject(3857, f'{output_directory}/vector/HV_line')

In [20]:
nepal.distance_to_electricity(hv_lines=hv_lines, mv_lines=mv_lines)

In [21]:
path = r"data\nighttime_lights.tif"
os.makedirs(f'{output_directory}rasters/Night_lights', exist_ok=True)
ntl = RasterLayer('rasters', 'Night_lights', layer_path=path, resample='average')
ntl.align(population.path, f'{output_directory}/rasters/Night_lights')
ntl.layer[np.isnan(ntl.layer)] = 0 # hey lets make sure that we interpolate NaNs

nepal.raster_to_dataframe(ntl.layer, name='Night_lights', method='read')

In [22]:
nepal.elec_current()

We have identified the existence of transformers or MV lines as input data; therefore we proceed using those for the calibration


In [23]:
# nepal.gdf.crs = 3857
# nepal.gdf.to_file('nepal/results.gpkg', driver='GPKG')

In [24]:
nepal.gdf['Elec_pop_calib'].sum() / nepal.gdf['Calibrated_pop'].sum()

0.5201280449140765

In [25]:
nepal.techs['Electricity'].morbidity(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)

In [26]:
nepal.techs['Electricity'].morbidity

18788130.060000986

In [27]:
nepal.techs['Electricity'].mortality(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)

In [28]:
nepal.techs['Electricity'].mortality

427020517.52045035

In [29]:
nepal.techs['Electricity'].deaths_avoided

13653.562187748523

In [30]:
nepal.techs['Electricity'].carbon_emissions(nepal.specs, nepal.gdf, nepal.base_fuel.carbon)

In [31]:
nepal.techs['Electricity'].decreased_carbon_emissions

0         1.086395
1         1.629592
2         0.192308
3         0.543197
4         0.543197
           ...    
96717     3.416899
96718    18.697766
96719    10.998686
96720    22.405873
96721     4.691313
Name: Households, Length: 96722, dtype: float64

In [32]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Demand\Wealth Index\Wealth index 2011.tif"
wealth_index = RasterLayer('rasters', 'Wealth_index', layer_path=path,  resample='average')
os.makedirs(f'{output_directory}/rasters/Wealth_index', exist_ok=True)
interpolate(wealth_index.path)
wealth_index.align(population.path, f'{output_directory}/rasters/Wealth_index')

In [33]:
nepal.get_value_of_time(wealth_index)

In [34]:
nepal.techs['Electricity'].time_saved(nepal.gdf, nepal.specs)

In [35]:
nepal.techs['Electricity'].time_value.sum()

1076537896.6564708

In [36]:
nepal.techs['Electricity'].total_time_saved.sum()

149663438.6670127

In [37]:
nepal.gdf

,geometry,Pop,Calibrated_pop,IsUrban,Households,base_fuel_time,base_fuel_travel_time,HV_line_dist,MV_line_dist,Night_lights,Elec_pop_calib,Current_elec,Elec_dist,Final_Elec_Code2020,value_of_time
0,POINT (9090104.530 3543192.836),31.171455,31.344224,0,5.498987,2993.471554,3.001292,58.872746,40.521599,0.000000,0.000000,0,40.521599,99,0.058780
1,POINT (9090104.530 3536192.836),46.757183,47.016337,0,8.248480,3442.932033,4.232691,52.392746,33.837849,0.000000,0.000000,0,33.837849,99,0.059025
2,POINT (9083104.530 3535192.836),5.517799,5.548382,0,0.973400,2239.061114,0.934414,54.708317,35.846897,0.000000,0.000000,0,35.846897,99,0.059683
3,POINT (9084104.530 3535192.836),15.585728,15.672112,0,2.749493,2280.672201,1.048417,54.203320,35.355339,0.000000,0.000000,0,35.355339,99,0.059637
4,POINT (9064104.530 3534192.836),15.585728,15.672112,0,2.749493,2972.034660,2.942561,64.536812,47.634022,0.000000,0.000000,0,47.634022,99,0.064742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96717,POINT (9724104.530 3044192.836),98.039602,98.582991,0,17.295262,7652.475230,15.765686,13.038404,1.414214,0.000000,98.582991,1,1.414214,1,0.099102
96718,POINT (9797104.530 3044192.836),536.486856,539.460361,1,94.642169,3599.900735,4.662742,17.029387,0.000000,0.000000,539.460361,1,0.000000,1,0.101375
96719,POINT (9798104.530 3044192.836),315.580504,317.329624,0,55.671864,3660.375916,4.828427,17.117243,1.000000,0.000000,317.329624,1,1.000000,1,0.100924
96720,POINT (9722104.530 3043192.836),642.881959,646.445163,1,113.411432,7919.425593,16.497056,11.000000,0.000000,0.891932,NaN,1,0.000000,1,0.104294


In [38]:
nepal.techs['Electricity'].discounted_om(nepal.gdf, nepal.specs)

In [39]:
nepal.techs['Electricity'].discounted_om_costs

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
96717    0.0
96718    0.0
96719    0.0
96720    0.0
96721    0.0
Name: Households, Length: 96722, dtype: float64

In [40]:
nepal.techs['Electricity'].discounted_inv(nepal.gdf, nepal.specs)

In [41]:
nepal.techs['Electricity'].discounted_investments

0         302.444271
1         453.666406
2          53.537020
3         151.222135
4         151.222135
            ...     
96717     951.239385
96718    5205.319271
96719    3061.952512
96720    6237.628769
96721    1306.026717
Name: Households, Length: 96722, dtype: float64

In [42]:
nepal.techs['Electricity'].discounted_meals(nepal.gdf, nepal.specs)

In [43]:
nepal.techs['Electricity'].discounted_energy

0         25485.885291
1         38228.827936
2          4511.371144
3         12742.942645
4         12742.942645
             ...      
96717     80157.503996
96718    438633.436558
96719    258019.668564
96720    525622.425941
96721    110054.149870
Name: Households, Length: 96722, dtype: float64

In [44]:
nepal.techs['Electricity'].discount_fuel_cost(nepal.gdf, nepal.specs)

In [45]:
nepal.techs['Electricity'].discounted_fuel_cost

0         417.685342
1         626.528013
2          73.936360
3         208.842671
4         208.842671
            ...     
96717    1313.692427
96718    7188.714655
96719    4228.655679
96720    8614.367536
96721    1803.665234
Length: 96722, dtype: float64

In [46]:
nepal.techs['Electricity'].salvage(nepal.gdf, nepal.specs)

In [47]:
nepal.techs['Electricity'].discounted_salvage_cost

0         272.199844
1         408.299765
2          48.183318
3         136.099922
4         136.099922
            ...     
96717     856.115446
96718    4684.787344
96719    2755.757261
96720    5613.865892
96721    1175.424045
Name: Households, Length: 96722, dtype: float64

In [48]:
nepal.techs['Electricity'].net_benefit(nepal.gdf)

In [49]:
nepal.gdf['net_benefit_biomass'] = 0

In [50]:
nepal.maximum_net_benefit()

In [51]:
nepal.gdf.loc[nepal.gdf['final_tech']=='electricity']

,geometry,Pop,Calibrated_pop,IsUrban,Households,base_fuel_time,base_fuel_travel_time,HV_line_dist,MV_line_dist,Night_lights,Elec_pop_calib,Current_elec,Elec_dist,Final_Elec_Code2020,value_of_time,net_benefit_electricity,net_benefit_biomass,final_tech,maximum_net_benefit
104,POINT (9100104.530 3509192.836),142.562189,143.352347,0,25.149534,3064.983407,3.197215,23.706539,5.099020,0.000000,143.352347,1,5.099020,1,0.058639,3499.079066,0,electricity,3499.079066
113,POINT (9099104.530 3508192.836),53.992006,54.291259,0,9.524782,2132.696487,0.643004,23.323807,4.472136,0.000000,54.291259,1,4.472136,1,0.058688,805.087206,0,electricity,805.087206
114,POINT (9102104.530 3508192.836),66.267554,66.634846,0,11.690324,3130.244750,3.376013,21.931713,4.000000,0.000000,66.634846,1,4.000000,1,0.058614,1670.544668,0,electricity,1670.544668
122,POINT (9099104.530 3507192.836),151.177617,152.015526,0,26.669390,2313.338564,1.137914,22.472204,3.605551,0.000000,152.015526,1,3.605551,1,0.058662,2535.945023,0,electricity,2535.945023
123,POINT (9101104.530 3507192.836),165.668886,166.587114,0,29.225809,2692.435970,2.176537,21.470911,3.000000,0.000000,166.587114,1,3.000000,1,0.058649,3428.255471,0,electricity,3428.255471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96717,POINT (9724104.530 3044192.836),98.039602,98.582991,0,17.295262,7652.475230,15.765686,13.038404,1.414214,0.000000,98.582991,1,1.414214,1,0.099102,11839.478784,0,electricity,11839.478784
96718,POINT (9797104.530 3044192.836),536.486856,539.460361,1,94.642169,3599.900735,4.662742,17.029387,0.000000,0.000000,539.460361,1,0.000000,1,0.101375,27374.792738,0,electricity,27374.792738
96719,POINT (9798104.530 3044192.836),315.580504,317.329624,0,55.671864,3660.375916,4.828427,17.117243,1.000000,0.000000,317.329624,1,1.000000,1,0.100924,16372.792758,0,electricity,16372.792758
96720,POINT (9722104.530 3043192.836),642.881959,646.445163,1,113.411432,7919.425593,16.497056,11.000000,0.000000,0.891932,NaN,1,0.000000,1,0.104294,84815.254109,0,electricity,84815.254109


In [52]:
nepal.gdf

,geometry,Pop,Calibrated_pop,IsUrban,Households,base_fuel_time,base_fuel_travel_time,HV_line_dist,MV_line_dist,Night_lights,Elec_pop_calib,Current_elec,Elec_dist,Final_Elec_Code2020,value_of_time,net_benefit_electricity,net_benefit_biomass,final_tech,maximum_net_benefit
0,POINT (9090104.530 3543192.836),31.171455,31.344224,0,5.498987,2993.471554,3.001292,58.872746,40.521599,0.000000,0.000000,0,40.521599,99,0.058780,-99999.000000,0,biomass,0.000000
1,POINT (9090104.530 3536192.836),46.757183,47.016337,0,8.248480,3442.932033,4.232691,52.392746,33.837849,0.000000,0.000000,0,33.837849,99,0.059025,-99999.000000,0,biomass,0.000000
2,POINT (9083104.530 3535192.836),5.517799,5.548382,0,0.973400,2239.061114,0.934414,54.708317,35.846897,0.000000,0.000000,0,35.846897,99,0.059683,-99999.000000,0,biomass,0.000000
3,POINT (9084104.530 3535192.836),15.585728,15.672112,0,2.749493,2280.672201,1.048417,54.203320,35.355339,0.000000,0.000000,0,35.355339,99,0.059637,-99999.000000,0,biomass,0.000000
4,POINT (9064104.530 3534192.836),15.585728,15.672112,0,2.749493,2972.034660,2.942561,64.536812,47.634022,0.000000,0.000000,0,47.634022,99,0.064742,-99999.000000,0,biomass,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96717,POINT (9724104.530 3044192.836),98.039602,98.582991,0,17.295262,7652.475230,15.765686,13.038404,1.414214,0.000000,98.582991,1,1.414214,1,0.099102,11839.478784,0,electricity,11839.478784
96718,POINT (9797104.530 3044192.836),536.486856,539.460361,1,94.642169,3599.900735,4.662742,17.029387,0.000000,0.000000,539.460361,1,0.000000,1,0.101375,27374.792738,0,electricity,27374.792738
96719,POINT (9798104.530 3044192.836),315.580504,317.329624,0,55.671864,3660.375916,4.828427,17.117243,1.000000,0.000000,317.329624,1,1.000000,1,0.100924,16372.792758,0,electricity,16372.792758
96720,POINT (9722104.530 3043192.836),642.881959,646.445163,1,113.411432,7919.425593,16.497056,11.000000,0.000000,0.891932,NaN,1,0.000000,1,0.104294,84815.254109,0,electricity,84815.254109
